In [ ]:
# for training av validation
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

import time
import numpy as np

import sklearn.metrics

from vocparseclslabels import PascalVOC

from PIL import Image
# custom functions
from vocTrainEval import dataset_voc, evaluate_meanavgprecision, traineval2_model_nocv, train_epoch
# for plotting and stuff
import matplotlib.pyplot as plt
# GUI stuff
import PySimpleGUI as sg
from io import BytesIO

In [ ]:
root_dir = "VOCdevkit/VOC2012/"
pVOC = PascalVOC(root_dir) # object for getting the correct file names
classes = pVOC.list_image_sets()

best_all = torch.load("models/bestMdl.pt")
fnames = np.array(best_all["fnames"])
labels = best_all["labels"].cpu().numpy().astype(int)
preds = best_all["preds"].cpu().numpy()

img_dir = "VOCdevkit/VOC2012/JPEGImages/"

fnames[labels[:, 0]]

In [ ]:
best_all.keys()

In [ ]:
def getPNGimage(img_dir, class_name, rank, fnames, preds, classes):
    class_nr = classes.index(class_name)
    sort_inds = preds[:, class_nr].argsort()
    
    fname = fnames[sort_inds][-1 - rank]
    
    filename = img_dir + fname + ".jpg"
    
    im = Image.open(filename)
    bio = BytesIO()
    im.save(bio, format='PNG')
    return bio.getvalue()

In [ ]:
sg.theme("DarkAmber")   # Add a touch of color
# All the stuff inside your window.

img = sg.Image(data = getPNGimage(img_dir, classes[0], 0, fnames, preds, classes), key = "image")

layout = [  [sg.Button("<-----"), sg.Text('0', key = "rank", size=(5, 1), justification="center"), sg.Button("----->"), sg.Button("Close")],
            [sg.Combo(classes, size=(15, 20), default_value = classes[0], enable_events=True, key="combo")],
            [img] ]

# Create the Window
window = sg.Window("Window Title", layout)
rank = 0
# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == "Close": # if user closes window or clicks cancel
        break
    if event == "<-----":
        rank -= 1
    if event == "----->":
        rank += 1
    if event == "combo":
        rank = 0
        
    window["rank"].update(str(rank))
    window["image"].update(data = getPNGimage(img_dir, values["combo"], rank, fnames, preds, classes))

window.close()

In [1]:
# for training av validation
import torch

import numpy as np
from vocparseclslabels import PascalVOC

from PIL import Image
# GUI stuff
import PySimpleGUI as sg
from io import BytesIO

root_dir = "VOCdevkit/VOC2012/"
pVOC = PascalVOC(root_dir) # object for getting the correct file names
classes = pVOC.list_image_sets()

best_all = torch.load("models/bestMdl.pt")
fnames = np.array(best_all["fnames"])
labels = best_all["labels"].cpu().numpy().astype(int)
preds = best_all["preds"].cpu().numpy()

img_dir = "VOCdevkit/VOC2012/JPEGImages/"

def getPNGimage(img_dir, class_name, rank, fnames, preds, classes):
    class_nr = classes.index(class_name)
    sort_inds = preds[:, class_nr].argsort()
    
    fname = fnames[sort_inds][-1 - rank]
    
    filename = img_dir + fname + ".jpg"
    
    im = Image.open(filename)
    bio = BytesIO()
    im.save(bio, format='PNG')
    return bio.getvalue()

# ------------------- GUI -----------------------------

sg.theme("DarkAmber")   # Add a touch of color
# All the stuff inside your window.

img = sg.Image(data = getPNGimage(img_dir, classes[0], 0, fnames, preds, classes), key = "image")

layout = [  [sg.Button("<-----"), sg.Text('0', key = "rank", size=(5, 1), justification="center"), sg.Button("----->"), sg.Button("Close")],
            [sg.Combo(classes, size=(15, 20), default_value = classes[0], enable_events=True, key="combo")],
            [img] ]

# Create the Window
window = sg.Window("Window Title", layout)
rank = 0
# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == "Close": # if user closes window or clicks cancel
        break
    if event == "<-----":
        rank -= 1
    if event == "----->":
        rank += 1
    if event == "combo":
        rank = 0
        
    window["rank"].update(str(rank))
    window["image"].update(data = getPNGimage(img_dir, values["combo"], rank, fnames, preds, classes))

window.close()